# Lecture 2: The classical time evolution operator and numerical integrators

## Physics 7810, Spring 2020

## 2.1 - Original derivation of the Verlet algorithm 

The original derivation of the Verlet algorithm introduced in the previous lecture made use of Taylor expansions to construct an explicitly time-reversible integration scheme. One first writes the 'forward' Taylor expansion to obtain the position of a particle at time $t + \Delta t$ from the position, velocity, and acceleration at time $t$:

$${\bf r}(t + \Delta t) = {\bf r}(t) + \Delta t {\bf v}(t) + \frac{1}{2} \Delta t^2 {\bf a}(t) + ...$$

Similarly, one can write the 'reverse' Taylor expansion:

$${\bf r}(t - \Delta t) = {\bf r}(t) - \Delta t {\bf v}(t) + \frac{1}{2} \Delta t^2 {\bf a}(t) - ...$$


Adding these two expansions and rearranging terms results in the original Verlet algorithm:

$${\bf r}(t + \Delta t) = 2 {\bf r}(t) - {\bf r}(t - \Delta t) + \Delta t^2 {\bf a}(t)
= 2 {\bf r}(t) - {\bf r}(t - \Delta t) + \frac{\Delta t^2}{m} {\bf f}[{\bf r}(t)],$$

where we have made use of Newton's second law in the final step. Note that the terms involving the velocity vanish, as do all odd-order terms in the Taylor expansion, so the leading error term is of order $\Delta t^4$ (the local error). Following the usual convention for ODE solvers, the Verlet position update has order 3 (a method is said to be of order $p$ is the leading error term is of order $\Delta t^{p+1}$).

This integrator involves only the particle position, and does not require the velocity. If the velocities are required, however (e.g., to calculate the kinetic energy or temperature) they can be calculated from the positions:

$${\bf v}(t) = \frac{{\bf r}(t + \Delta t) - {\bf r}(t - \Delta t)}{2 \Delta t}.$$

The leading error term in this expression for the velocity is $O(\Delta t^3)$, so the Verlet velocity update has order 2.

It's easy to show that the Verlet algorithm is equivalent to the velocity Verlet method introduced previously, and to other Verlet variants (e.g., 'position' Verlet). The Verlet algorithms are time-reversible by construction, are remarkably stable, and lead to good energy conservation over long integration times ('fidelity'). These desirable properties result from the fact that the Verlet methods preserve certain properties of the the exact Hamiltonian dynamics, as we now discuss.

In [3]:
from IPython.display import Video

In [7]:
Video("movies/vdist_whitebackground.mov",width=1000)

The time evolution is completely deterministic, but rapidly leads to a 'random' Maxwellian distribution of velocities.

Animation courtesy of Robert Blackwell.

In [8]:
Video("movies/vdist_lowdens_whitebackground.mov",width=1000)

At low densities, long-wavelength density fluctuations (sound waves) persist long after the Maxwellian velocity distribution is established.

Animation courtesy of Robert Blackwell.

## 2.2 - Liouville formulation of time-reversible algorithms 

Stable numerical integration schemes can be constructed explicitly by preserving formal properties of the underlying continuous-time Hamiltonian dynamics.

Consider the time evolution of any function $A(p^{3N},q^{3N})$ of the phase space momenta $p^{3N}$ and coordinates $q^{3N}$. The time derivative of $A$ is

$$
\frac{dA}{dt} = \sum_{\alpha = 1}^{3N} \left[ \frac{\partial A}{\partial q_\alpha} \frac{d q_\alpha}{d t} + \frac{\partial A}{\partial p_\alpha} \frac{d p_\alpha}{d t} \right]
$$
$$
= \sum_{\alpha = 1}^{3N} \left[ \frac{\partial A}{\partial q_\alpha} \frac{\partial H}{\partial p_\alpha} - \frac{\partial A}{\partial p_\alpha} \frac{\partial H}{\partial q_\alpha} \right]
= [A, H]
$$

where $H(p^{3N},q^{3N})$ is the Hamiltonian, and Hamilton's equations of motion $d q_\alpha / d t = \partial H / \partial p_\alpha$ and
$d p_\alpha / d t = - \partial H / \partial q_\alpha$ have been used for the time derivatives in the second step.


Here we have defined the *Poisson bracket*,

$$
[A, H] = \sum_{\alpha = 1}^{3N} \left[ \frac{\partial A}{\partial q_\alpha} \frac{\partial H}{\partial p_\alpha} - \frac{\partial A}{\partial p_\alpha} \frac{\partial H}{\partial q_\alpha} \right]
$$

which is the classical analog of the quantum mechanical commutator. The Poisson bracket generates the time evolution of $A$, allowing us to introduce a time evolution operator that acts on any phase space function $A$, defined by

$$
i L A = [A, H]
$$

where $iL$ is known as the *Liouville operator*.

The Liouville operator can be written as a differential operator,

$$
iL = \sum_{\alpha = 1}^{3N} \left[ \frac{\partial H}{\partial p_\alpha} \frac{\partial}{\partial q_\alpha} - \frac{\partial H}{\partial q_\alpha} \frac{\partial}{\partial p_\alpha} \right],
$$

and the differential equation $dA/dt = i L A$ can be solved formally to obtain

$$
A[p^{3N}(t),q^{3N}(t)] = e^{iLt} A[p^{3N}(0),q^{3N}(0)]
$$

where the derivatives in $iL$ act on the initial phase space variables $p^{3N}(0),q^{3N}(0)$.

The operator $\exp(iLt)$, known as the *classical propagator*, strongly resembles the quantum propagator $\exp(-i\hat{H}t/\hbar)$, where $\hat{H}$ is the Hamiltonian operator; this is why the factor of $i$ is included in the definition of the Liouville operator.

The operator $L$ can be shown to be Hermitian, so the classical propagator $\exp(iLt)$ is a unitary operator on the phase space (the Jacobian of the variable transformation generated by the dynamics is equal to one). This *area preserving* property of the dynamics turns out to be the critical property that any good discrete-time integrator of a Hamiltonian system should possess.

The function $A$ can be taken to be the $6N$-dimensional vector of phase space variables $\Gamma(t) = p^{3N}(t),q^{3N}(t)$. In this case, the formal solution to Hamilton's equations of motion is

$$
\Gamma(t) = e^{iLt} \Gamma(0)
$$

This formal solution isn't very useful as it stands, because it's usually impossible to calculate the action of the operator $e^{iLt}$ on $\Gamma(0)$ directly. However, it *is* useful for developing approximate numerical methods for solving the equations of motion.

First, note that the Liouville operator can be written as the sum of two contributions that act on coordinates and momenta separately

$$
iL = iL_q+iL_p,
$$

where

$$
iL_q = \sum_{\alpha = 1}^{3N} \frac{\partial H}{\partial p_\alpha} \frac{\partial}{\partial q_\alpha}
= \sum_{\alpha = 1}^{3N} \dot{q}_\alpha \frac{\partial}{\partial q_\alpha}
$$

and

$$
iL_p = - \sum_{\alpha = 1}^{3N} \frac{\partial H}{\partial q_\alpha} \frac{\partial}{\partial p_\alpha}
= \sum_{\alpha = 1}^{3N} \dot{p}_\alpha \frac{\partial}{\partial p_\alpha}
= \sum_{\alpha = 1}^{3N} f_\alpha \frac{\partial}{\partial p_\alpha},
$$

and where $f_\alpha$ is the force acting on degree of freedom $\alpha$.

The differential operators $iL_q$ and $iL_p$ are *noncommuting*. Given a function $A(\Gamma)$,

$$
iL_q iL_p A(\Gamma) \neq  iL_p iL_q A(\Gamma).
$$
 
In other words, the order in which the operators are applied is important.
 
Equivalently, we can say that the *commutator* of the two operators, defined as

$$
[iL_q,iL_p] \equiv iL_q iL_p - iL_p iL_q
$$

is nonzero.
 

The fact that $iL_q$ and $iL_p$ don't commute can be illustrated by a simple one-dimensional example. Consider the Hamiltonian

$$
H = \frac{p^2}{2 m} + U(x).
$$

In this case,

$$
iL_x = \frac{p}{m} \frac{\partial}{\partial x}
$$

and

$$
iL_p = f(x) \frac{\partial}{\partial p},
$$

where $f(x) = - dU(x) / dx$.

The action of $iL_x iL_p$ on an arbitrary function $A(x,p)$ is

$$
iL_x iL_p = \frac{p}{m} \frac{\partial}{\partial x} f(x) \frac{\partial}{\partial p} A(x,p)
= \frac{p}{m} f(x) \frac{\partial^2 A}{\partial p \partial x} + \frac{p}{m} f^\prime(x) \frac{\partial A}{\partial p},
$$

and the action of $iL_p iL_x$ on $A(x,p)$ is
$$
iL_p iL_x = f(x) \frac{\partial}{\partial p} \frac{p}{m} \frac{\partial}{\partial x} A(x,p)
= f(x) \frac{p}{m} \frac{\partial^2 A}{\partial p \partial x} + f(x) \frac{1}{m} \frac{\partial A}{\partial x},
$$

Thus,

$$
\left[ iL_x, iL_p \right] A(x,p) = \frac{p}{m} f^\prime(x) \frac{\partial A}{\partial p} - \frac{f(x)}{m} \frac{\partial A}{\partial x}.
$$

Because the function $A(x,p)$ is arbitrary, this implies that the commutator is nonzero, and is given by

$$
\left[ iL_x, iL_p \right] = \frac{p}{m} f^\prime(x) \frac{\partial}{\partial p} - \frac{f(x)}{m} \frac{\partial}{\partial x}.
$$

The fact that $[ iL_x, iL_p ] \neq 0$ arises because Hamilton's equations of motion are coupled via the potential $U(x)$ (or force $f(x)$). The commutator would be zero for a noninteracting system (which wouldn't be very interesting).

Because $iL_q$ and $iL_p$ don't commute, the classical propagator $\exp(iLt) = \exp[(iL_q + iL_p)t]$ can't be factored into a simple product $\exp(iL_qt) \exp(iL_pt)$.
 
This is unfortunate, because the action of the individual operators $\exp(iL_qt)$ and $\exp(iL_pt)$ can be evaluated exactly in many cases.
 
It would be useful if the propagator could be expressed in terms of these two factors. In fact, there is a way to do this based on the *Trotter theorem*, which we explore shortly.

First, however, we will explain how exponential differential operators such as $\exp(iL_qt)$ and $\exp(iL_pt)$ act on functions of the phase space variables.

The action of an exponential differential operator is *defined* as an expansion of the exponential in a Taylor series

$$
\exp \left( a \frac{\partial}{\partial x} \right) f(x) \equiv \sum_{n = 0}^\infty \frac{1}{n!} \left( a \frac{\partial}{\partial x} \right)^n f(x)
= \sum_{n = 0}^\infty \frac{1}{n!} a^n \frac{\partial^n f(x)}{\partial x^n} = f(x + a),
$$

which results in a translation of $f(x)$ by $a$.
 
Thus,

$$
e^{iL_q \Delta t} \binom{q(0)}{p(0)} = e^{\Delta t \dot{q}(0) \frac{\partial}{\partial q}} \binom{q(0)}{p(0)}
= \binom{q(0) + \Delta t \dot{q}(0)}{p(0)}
= \binom{q(0) + \frac{\Delta t}{m} p(0)}{p(0)}
$$
 
 and
 
$$
e^{iL_p \Delta t} \binom{q(0)}{p(0)} = e^{\Delta t \dot{p}(0) \frac{\partial}{\partial p}} \binom{q(0)}{p(0)} = e^{\Delta t f(0) \frac{\partial}{\partial p}} \binom{q(0)}{p(0)}
= \binom{q(0)}{p(0) + \Delta t f(0)},
$$

where $q(0)$, $p(0)$, and $f(0)$ refer to the coordinates, momenta, and forces evaluated at $t = 0$.


We now consider an approximate splitting of the Liouville operator,

$$
e^{iLt} \approx e^{iL_p \Delta t} e^{iL_q \Delta t}.
$$

This is low-order approximation that lacks time-reversal symmetry, but it does preserve the area-preserving property of the exact time-evolution operator.

This splitting generates the following equations of motion:

$$
e^{iL \Delta t} \binom{q(0)}{p(0)} \approx e^{iL_p \Delta t} e^{iL_q \Delta t} \binom{q(0)}{p(0)}
$$

$$
= e^{iL_p \Delta t} \binom{q(0) + \frac{\Delta t}{m} p(0)}{p(0)}
= \binom{q(0) + \frac{\Delta t}{m} p(0)}{p(0) + \Delta t f(\Delta t)},
$$

where $f(\Delta t)$ is the force evaluated at $q(\Delta t) = q(0) + \frac{\Delta t}{m} p(0)$. This is the symplectic Euler algorithm.

To construct a time-reversible integrator, we consider a symmetric splitting of the time-evolution operator,

$$ e^{iL \Delta t} \binom{q(0)}{p(0)} \approx e^{iL_p \Delta t/2} e^{iL_q \Delta t} e^{iL_p \Delta t/2} \binom{q(0)}{p(0)}
$$

$$
= e^{iL_p \Delta t/2} e^{iL_q \Delta t} \binom{q(0)}{p(0) + \frac{\Delta t}{2} f(0)}
= e^{iL_p \Delta t/2} \binom{q(0) + \frac{\Delta t}{m} p(0) + \frac{\Delta t^2}{2 m} f(0)}{p(0) + \frac{\Delta t}{2} f(0)}
$$

$$
= \binom{q(0) + \frac{\Delta t}{m} p(0) + \frac{\Delta t^2}{2 m} f(0)}{p(0) + \frac{\Delta t}{2} [ f(0) + f(\Delta t)]},
$$

Where $f(\Delta t)$ is the force evaluated at $q(\Delta t) = q(0) + \frac{\Delta t}{m} p(0) + \frac{\Delta t^2}{2 m} f(0)$. This is the velocity Verlet algorithm.


## 2.2 - Trotter theorem

The foregoing development of time-reversible integration methods can be put on a more rigorous footing using the symmetric Trotter theorem, which states that for two operators $A$ and $B$ for which $[A,B] \neq 0$

$$
 e^{A+B} = \lim_{P \to \infty} \left[ e^{B/2P} e^{A/P} e^{B/2P} \right]^P
$$

where $P$ is an integer. The Trotter formula is a corollary of the *Baker-Campbell-Hausdorff* identity.

Applying the Trotter theorem to the classical propagator yields

$$
e^{iLt} = e^{(iL_q+iL_p)t} = \lim_{P \to \infty} \left[ e^{iL_pt/2P} e^{iL_qt/P} e^{iL_pt/2P} \right]^P
$$

$$
 = \lim_{\Delta t \to 0} \left[ e^{iL_p \Delta t/2} e^{iL_q \Delta t} e^{iL_p \Delta t/2} \right]^{t / \Delta t}
$$
where $\Delta t = t / P$

For finite $P$, the Trotter formula leads to an approximation for $\exp(iLt)$,

$$
e^{iLt} \approx \left[ e^{iL_p \Delta t/2} e^{iL_q \Delta t} e^{iL_p \Delta t/2} \right]^{t / \Delta t} + O(P \Delta t^3),
$$

where the leading order error is of order $P \Delta t^3$.
 
According to this equation, an approximate (discrete time) propagation can be generated by performing $P$ steps of *finite* length $\Delta t$ using the factorized propagator

$$
e^{iL \Delta t} \approx e^{iL_p \Delta t/2} e^{iL_q \Delta t} e^{iL_p \Delta t/2} + O(\Delta t^3)
$$

for each step, which has *local* error of order $\Delta t^3$, so this method (the velocity Verlet algorithm) has order 2.
 
Because the individual operators are *unitary*, the discrete time dynamics satisfies Liouville's theorem, and phase space volume is conserved. Splitting the operator time evolution operator *symmetrically* ensures that the discrete time dynamics is *time reversible*, like the true continuous time Hamiltonian dynamics.

## 2.2 - Energy conservation and the shadow Hamiltonian

Why is long-term energy drift absent in the Verlet (and other symplectic) algorithms? This can be understood by analyzing the error introduced in the operator-splitting procedure, using the Baker-Campbell-Hausdorff identity, which leads to an exact expression for $\exp{iL \Delta t}$ in terms of an infinite series expansion in commutators of $iL_p$ and $iL_q$. If all $n$th-order commutators exist (i.e., if the Hamiltonian is an infinitely differentiable function of $p$ and $q$), then the error involved in the Verlet algorithm can be evaluated:

$$
e^{iL_p \Delta t/2} e^{iL_q \Delta t} e^{iL_p \Delta t/2} = e^{iL \Delta t + \epsilon}
$$

where $\epsilon$ is an operator that can be expressed in terms of commutators of $iL_p$ and $iL_q$,

$$
\epsilon = \sum_{n=1}^\infty (\Delta t)^{2 n + 1} c_{2n + 1},
$$

where $c_m$ denotes a combination of $m$th-order commutators.

For example, the leading term in this series is:

$$
-(\Delta t)^3 \left( \frac{1}{24} [iL_q,[iL_q,iL_p]] + \frac{1}{12} [iL_p,[iL_q,iL_p]] \right)
$$

Amazingly, *if this series converges*, then we can define a *shadow Liouville operator*

$$
iL_{\rm shadow} \equiv iL + \frac{1}{\Delta t} \epsilon
$$

that corresponds to a *shadow Hamiltonian* $H_{\rm shadow}$ that is *exactly conserved* by the symplectic discrete-time dynamics. The difference between the conserved shadow Hamiltonian and the true Hamiltonian is of order $\Delta t^n$ *for all times*, where $n$ depends on the order of the algorithm.